<a href="https://colab.research.google.com/github/uozumi-n/kaigisitu_iio/blob/main/%E7%9B%B4%E5%96%B6%E5%A3%B2%E4%B8%8A%E7%AE%A1%E7%90%86%E3%83%AC%E3%83%9D%E3%83%BC%E3%83%88%E4%BD%9C%E6%88%90%E3%83%84%E3%83%BC%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ①インスタベース利用データ集計プログラム（最終確定版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone

def perform_aggregation(df):
    """インスタベースのデータフレームを受け取り、集計を実行"""
    # 1. データのクリーニング（集計対象：予約金額 (税込)）
    target_sales_col = '予約金額 (税込)'

    if target_sales_col in df.columns:
        def clean_currency(x):
            if pd.isna(x): return 0
            s = re.sub(r'[^0-9.-]', '', str(x))
            return pd.to_numeric(s, errors='coerce')
        df['売上'] = df[target_sales_col].apply(clean_currency).fillna(0)
    else:
        raise ValueError(f"カラム '{target_sales_col}' が見つかりません。")

    # 【重要】日付変換を強化（format='mixed' を使用）
    df['開始'] = pd.to_datetime(df['利用開始日時'], errors='coerce', format='mixed')
    df['終了'] = pd.to_datetime(df['利用終了日時'], errors='coerce', format='mixed')
    df['利用時間_td'] = df['終了'] - df['開始']
    df['集計日'] = df['開始'].dt.normalize()

    # 無効な行を除去
    df = df.dropna(subset=['集計日', '利用時間_td'])

    # 2. グルーピング集計
    group_keys = ['集計日', '施設名', 'スペース名']
    summary = df.groupby(group_keys, dropna=False).agg(
        件数=('予約ID', 'size'),
        利用時間_合計=('利用時間_td', 'sum'),
        売上_合計=('売上', 'sum')
    ).reset_index()

    # 3. 期間フィルタリング（昨日以前）
    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    summary = summary[summary['集計日'].dt.date <= yesterday_in_japan].copy()

    # 4. フォーマット変換
    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    summary['月次タグ'] = summary['集計日'].dt.strftime('%Y年%m月')
    summary['日付'] = summary['集計日'].dt.strftime('%Y-%m-%d')
    summary['利用時間'] = summary['利用時間_合計'].apply(timedelta_to_hhmm)
    summary['売上'] = summary['売上_合計'].round().astype(int)
    summary['設備名'] = summary['スペース名']
    summary['プラン名'] = summary['スペース名']

    return summary[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上']]

def load_csv(uploaded_file):
    filename = next(iter(uploaded_file))
    try:
        return pd.read_csv(io.BytesIO(uploaded_file[filename]), encoding='utf-8-sig')
    except:
        return pd.read_csv(io.BytesIO(uploaded_file[filename]), encoding='cp932')

try:
    print("【手順1】タグ設置リストをアップロードしてください。")
    up_tag = files.upload()
    df_tag_master = load_csv(up_tag)
    df_mapping = df_tag_master[df_tag_master['媒体名'] == 'instabase'].copy()
    df_mapping = df_mapping[['媒体側施設名', '媒体側設備名', '統一施設名', '統一設備名']].drop_duplicates()
    df_mapping = df_mapping.rename(columns={'媒体側施設名': 'f_key', '媒体側設備名': 'e_key', '統一施設名': '統一店名'})

    print("\n【手順2】インスタベースの利用データをアップロードしてください。")
    up_data = files.upload()
    df_input = load_csv(up_data)

    res_df = perform_aggregation(df_input)

    # 照合
    res_df['f_tmp'] = res_df['施設名'].astype(str).str.strip()
    res_df['e_tmp'] = res_df['設備名'].astype(str).str.strip()
    df_mapping['f_tmp'] = df_mapping['f_key'].astype(str).str.strip()
    df_mapping['e_tmp'] = df_mapping['e_key'].astype(str).str.strip()

    merged_df = pd.merge(res_df, df_mapping[['f_tmp', 'e_tmp', '統一店名', '統一設備名']], on=['f_tmp', 'e_tmp'], how='left')
    unmatched = merged_df[merged_df['統一店名'].isna()][['施設名', '設備名']].drop_duplicates()

    merged_df['統一店名'] = merged_df['統一店名'].fillna('')
    merged_df['統一設備名'] = merged_df['統一設備名'].fillna('')

    out_name = 'インスタベース集計結果_統合版.csv'
    merged_df.drop(columns=['f_tmp', 'e_tmp']).to_csv(out_name, index=False, encoding='utf-8-sig')
    files.download(out_name)
    if not unmatched.empty:
        unmatched.to_csv('紐付け失敗リスト_インスタ.csv', index=False, encoding='utf-8-sig')
        files.download('紐付け失敗リスト_インスタ.csv')
    print("\n✅ 完了しました。")
except Exception as e:
    print(f"\n❌ エラー: {e}")

【手順1】タグ設置リストをアップロードしてください。


Saving 会議室シート_改 - タグ設置リスト (4).csv to 会議室シート_改 - タグ設置リスト (4).csv

【手順2】インスタベースの利用データをアップロードしてください。


Saving 他媒体データ置き場 - instabase のコピー (3).csv to 他媒体データ置き場 - instabase のコピー (3).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ 完了しました。


In [ ]:
#@title ②スペースマーケット売上明細集計プログラム（最終確定版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone

def perform_aggregation(df):
    """スペースマーケットのデータフレームを受け取り、集計を実行"""
    # 1. データのクリーニング（集計対象：成約金額）
    target_sales_col = '成約金額'

    if target_sales_col in df.columns:
        def clean_currency(x):
            if pd.isna(x): return 0
            s = re.sub(r'[^0-9.-]', '', str(x))
            return pd.to_numeric(s, errors='coerce')
        df['売上'] = df[target_sales_col].apply(clean_currency).fillna(0)
    else:
        raise ValueError(f"カラム '{target_sales_col}' が見つかりません。")

    # 日付変換を強化（format='mixed'）
    df['集計日'] = pd.to_datetime(df['実施日'], errors='coerce', format='mixed')

    if '利用開始時間' in df.columns and '利用終了時間' in df.columns:
        # 時間の計算も安全に行う
        df['利用時間_td'] = pd.to_datetime(df['利用終了時間'], format='mixed') - pd.to_datetime(df['利用開始時間'], format='mixed')
    else:
        df['利用時間_td'] = pd.Timedelta(0)

    df = df.dropna(subset=['集計日'])

    # 2. グルーピング集計
    group_keys = ['集計日', '施設名', 'プラン名', 'スペース名']
    summary = df.groupby(group_keys, dropna=False).agg(
        件数=('予約ID', 'size'),
        利用時間_合計=('利用時間_td', 'sum'),
        売上_合計=('売上', 'sum')
    ).reset_index()

    # 3. 期間フィルタリング（昨日以前）
    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    summary = summary[summary['集計日'].dt.date <= yesterday_in_japan].copy()

    # 4. フォーマット変換
    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    summary['月次タグ'] = summary['集計日'].dt.strftime('%Y年%m月')
    summary['日付'] = summary['集計日'].dt.strftime('%Y-%m-%d')
    summary['利用時間'] = summary['利用時間_合計'].apply(timedelta_to_hhmm)
    summary['売上'] = summary['売上_合計'].round().astype(int)

    # 【重要】プラン名とスペース名の入れ替え
    # 出力「プラン名」＝元の「スペース名」
    # 出力「設備名」＝元の「プラン名」（マスタの「媒体側設備名」と照合するため）
    summary['orig_plan'] = summary['プラン名']
    orig_p = summary['プラン名'].copy()
    orig_s = summary['スペース名'].copy()
    summary['プラン名'] = orig_s
    summary['設備名'] = orig_p

    return summary[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上', 'orig_plan']]

def load_csv_with_encoding(uploaded_file_dict):
    """文字コードを自動判定して読み込むヘルパー関数"""
    filename = next(iter(uploaded_file_dict))
    content = uploaded_file_dict[filename]
    try:
        # まずは UTF-8 で試行
        return pd.read_csv(io.BytesIO(content), encoding='utf-8-sig')
    except UnicodeDecodeError:
        # 失敗したら Shift-JIS で試行
        return pd.read_csv(io.BytesIO(content), encoding='cp932')

# --- メイン実行 ---
try:
    # 1. マスタの読み込み
    print("【手順1】タグ設置リストをアップロードしてください。")
    up_tag = files.upload()
    df_tag_master = load_csv_with_encoding(up_tag)

    # マッピング準備
    df_mapping = df_tag_master[df_tag_master['媒体名'] == 'spacemarket'].copy()
    df_mapping = df_mapping[['媒体側施設名', '媒体側設備名', '統一施設名', '統一設備名']].drop_duplicates()
    df_mapping = df_mapping.rename(columns={'媒体側施設名': 'f_key', '媒体側設備名': 'e_key', '統一施設名': '統一店名'})

    # 2. 売上明細の読み込み
    print("\n【手順2】スペースマーケットの売上明細をアップロードしてください。")
    up_data = files.upload()
    df_input = load_csv_with_encoding(up_data)

    # 3. 集計実行
    res_df = perform_aggregation(df_input)

    # 4. マスタ照合（一時キー作成）
    res_df['f_tmp'] = res_df['施設名'].astype(str).str.strip()
    res_df['p_tmp'] = res_df['orig_plan'].astype(str).str.strip()
    df_mapping['f_tmp'] = df_mapping['f_key'].astype(str).str.strip()
    df_mapping['p_tmp'] = df_mapping['e_key'].astype(str).str.strip()

    # 結合
    merged_df = pd.merge(res_df, df_mapping[['f_tmp', 'p_tmp', '統一店名', '統一設備名']], on=['f_tmp', 'p_tmp'], how='left')

    # 判定
    unmatched = merged_df[merged_df['統一店名'].isna()][['施設名', 'orig_plan']].drop_duplicates()

    # 空欄補完
    merged_df['統一店名'] = merged_df['統一店名'].fillna('')
    merged_df['統一設備名'] = merged_df['統一設備名'].fillna('')

    # 不要な一時カラムを削除して列順確定
    final_output = merged_df[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上', '統一店名', '統一設備名']]

    # 5. 出力
    out_name = 'スペースマーケット集計結果_統合版.csv'
    final_output.to_csv(out_name, index=False, encoding='utf-8-sig')
    files.download(out_name)

    if not unmatched.empty:
        unmatched.to_csv('紐付け失敗リスト_スペマ.csv', index=False, encoding='utf-8-sig')
        files.download('紐付け失敗リスト_スペマ.csv')
        print(f"⚠️ 照合できなかった項目があります。失敗リストを確認してください。")

    print("\n✅ 完了しました。")
except Exception as e:
    print(f"\n❌ エラー: {e}")

【手順1】タグ設置リストをアップロードしてください。


Saving 会議室シート_改 - タグ設置リスト.csv to 会議室シート_改 - タグ設置リスト (8).csv

【手順2】スペースマーケットの売上明細をアップロードしてください。


Saving 他媒体データ置き場 - spacemarket のコピー.csv to 他媒体データ置き場 - spacemarket のコピー (3).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⚠️ 照合できなかった項目があります。失敗リストを確認してください。

✅ 完了しました。


In [ ]:
#@title ③スペイシー利用データ集計プログラム（最終確定版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone

def perform_aggregation(df):
    """スペイシーのデータフレームを受け取り、集計を実行"""
    # 1. データのクリーニング（集計対象：差引合計売上金額（税込））
    target_sales_col = '差引合計売上金額（税込）'

    if target_sales_col in df.columns:
        def clean_currency(x):
            if pd.isna(x): return 0
            if isinstance(x, (int, float)): return x
            s = re.sub(r'[^0-9.-]', '', str(x))
            return pd.to_numeric(s, errors='coerce')
        df['売上'] = df[target_sales_col].apply(clean_currency).fillna(0)
    else:
        raise ValueError(f"カラム '{target_sales_col}' が見つかりません。")

    # ステータスフィルタリング
    if '予約ステータス' in df.columns:
        df = df[df['予約ステータス'] == '予約完了'].copy()

    # 日付変換を強化（format='mixed' を使用し、混合形式に対応）
    df['開始'] = pd.to_datetime(df['利用開始日時'], errors='coerce', format='mixed')
    df['終了'] = pd.to_datetime(df['利用終了日時'], errors='coerce', format='mixed')
    df['利用時間_td'] = df['終了'] - df['開始']
    df['集計日'] = df['開始'].dt.normalize()

    # 無効な行を除去
    df = df.dropna(subset=['集計日', '利用時間_td'])

    # 2. グルーピング集計
    group_keys = ['集計日', 'スペース名']
    summary = df.groupby(group_keys, dropna=False).agg(
        件数=('予約ID', 'size'),
        利用時間_合計=('利用時間_td', 'sum'),
        売上_合計=('売上', 'sum')
    ).reset_index()

    # 3. 期間フィルタリング（昨日以前）
    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    summary = summary[summary['集計日'].dt.date <= yesterday_in_japan].copy()

    # 4. フォーマット変換
    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    summary['月次タグ'] = summary['集計日'].dt.strftime('%Y年%m月')
    summary['日付'] = summary['集計日'].dt.strftime('%Y-%m-%d')
    summary['利用時間'] = summary['利用時間_合計'].apply(timedelta_to_hhmm)
    summary['売上'] = summary['売上_合計'].round().astype(int)

    # スペイシーの仕様に合わせてマッピング
    summary['施設名'] = summary['スペース名']
    summary['プラン名'] = summary['スペース名']
    summary['設備名'] = summary['スペース名']

    return summary[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上']]

def load_csv_with_encoding(uploaded_file_dict):
    """文字コードを自動判定して読み込むヘルパー関数"""
    filename = next(iter(uploaded_file_dict))
    content = uploaded_file_dict[filename]
    try:
        # まずは UTF-8 (BOM付き含む) で試行
        return pd.read_csv(io.BytesIO(content), encoding='utf-8-sig')
    except UnicodeDecodeError:
        # 失敗したら Shift-JIS (cp932) で試行
        return pd.read_csv(io.BytesIO(content), encoding='cp932')

# --- メイン実行 ---
try:
    # 1. マスタの読み込み
    print("【手順1】タグ設置リストをアップロードしてください。")
    up_tag = files.upload()
    df_tag_master = load_csv_with_encoding(up_tag)

    # マッピング準備（spacee）
    df_mapping = df_tag_master[df_tag_master['媒体名'] == 'spacee'].copy()
    df_mapping = df_mapping[['媒体側施設名', '媒体側設備名', '統一施設名', '統一設備名']].drop_duplicates()
    df_mapping = df_mapping.rename(columns={'媒体側施設名': 'f_key', '媒体側設備名': 'e_key', '統一施設名': '統一店名'})

    # 2. 利用データの読み込み
    print("\n【手順2】スペイシーの利用データをアップロードしてください。")
    up_data = files.upload()
    df_input = load_csv_with_encoding(up_data)

    # 3. 集計実行
    res_df = perform_aggregation(df_input)

    # 4. マスタ照合（設備名 ＝ スペイシーのスペース名 で照合）
    res_df['e_tmp'] = res_df['設備名'].astype(str).str.strip()
    df_mapping['e_tmp'] = df_mapping['e_key'].astype(str).str.strip()

    # 結合
    merged_df = pd.merge(res_df, df_mapping[['e_tmp', '統一店名', '統一設備名']], on=['e_tmp'], how='left')

    # 紐付け失敗チェック
    unmatched = merged_df[merged_df['統一店名'].isna()][['設備名']].drop_duplicates()

    # 空欄補完
    merged_df['統一店名'] = merged_df['統一店名'].fillna('')
    merged_df['統一設備名'] = merged_df['統一設備名'].fillna('')

    # 列順の確定
    final_output = merged_df[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上', '統一店名', '統一設備名']]

    # 5. 出力
    out_name = 'スペイシー集計結果_統合版.csv'
    final_output.to_csv(out_name, index=False, encoding='utf-8-sig')
    files.download(out_name)

    if not unmatched.empty:
        unmatched.to_csv('紐付け失敗リスト_スペイシー.csv', index=False, encoding='utf-8-sig')
        files.download('紐付け失敗リスト_スペイシー.csv')
        print(f"⚠️ マスタにないスペースがありました。失敗リストを確認してください。")

    print("\n✅ 完了しました。")
except Exception as e:
    print(f"\n❌ エラー: {e}")

【手順1】タグ設置リストをアップロードしてください。


Saving 会議室シート_改 - タグ設置リスト.csv to 会議室シート_改 - タグ設置リスト (10).csv

【手順2】スペイシーの利用データをアップロードしてください。


Saving 他媒体データ置き場 - spacee のコピー.csv to 他媒体データ置き場 - spacee のコピー (1).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⚠️ マスタにないスペースがありました。失敗リストを確認してください。

✅ 完了しました。


# **[いいアプリ元データの入手はこちらから](https://e-office.metabaseapp.com/public/dashboard/12bc87a0-0ae5-4302-9957-5b208cbd54ca)**<br>元データをダウンロードしたら次のステップに移ってください。

In [ ]:
#@title ④いいアプリ利用データ集計プログラム（最終確定版）

import pandas as pd
from google.colab import files
import io
import re
from datetime import datetime, timedelta, timezone

def perform_aggregation(df_main, df_ext):
    """通常分と延長分を合算し、昨日以前のデータを集計する関数"""
    MIN_SALES_THRESHOLD = 1

    # 強力な数値変換関数
    def clean_currency(x):
        if pd.isna(x): return 0
        if isinstance(x, (int, float)): return x
        s = re.sub(r'[^0-9.-]', '', str(x).replace('￥', '').replace('¥', '').replace(',', ''))
        return pd.to_numeric(s, errors='coerce')

    # 通常・延長それぞれの売上を数値化
    for df in [df_main, df_ext]:
        if '売上' in df.columns:
            df['売上'] = df['売上'].apply(clean_currency).fillna(0)

    # --- 1. 通常データの処理 ---
    df_main = df_main[df_main['売上'] >= MIN_SALES_THRESHOLD].copy()
    # 日付変換（format='mixed'で混合形式に対応）
    df_main['集計日'] = pd.to_datetime(df_main['売上確定日'], errors='coerce', format='mixed')

    # 時間計算用のクレンジング
    def clean_time_str(s):
        if pd.isna(s): return s
        return str(s).replace(', ', ' ').strip()

    start_t = pd.to_datetime(df_main['開始'].apply(clean_time_str), errors='coerce', format='mixed')
    end_act = pd.to_datetime(df_main['実際の終了時間'].apply(clean_time_str), errors='coerce', format='mixed')
    end_sch = pd.to_datetime(df_main['終了'].apply(clean_time_str), errors='coerce', format='mixed')

    df_main['利用時間_td'] = end_act.fillna(end_sch) - start_t
    df_main = df_main.dropna(subset=['集計日', '利用時間_td'])

    # --- 2. 延長データの処理 ---
    df_ext = df_ext[df_ext['売上'] >= MIN_SALES_THRESHOLD].copy()
    df_ext['集計日'] = pd.to_datetime(df_ext['売上確定日'], errors='coerce', format='mixed')
    df_ext['延長時間_td'] = pd.to_timedelta(df_ext['延長した分数'], unit='m', errors='coerce')
    df_ext = df_ext.dropna(subset=['集計日', '延長時間_td'])

    # --- 3. 集計と合算 ---
    group_keys = ['集計日', '施設名', 'プラン名', '設備名']

    summary_main = df_main.groupby(group_keys, dropna=False).agg(
        件数=('施設名', 'size'),
        通常時間_td=('利用時間_td', 'sum'),
        通常売上=('売上', 'sum')
    ).reset_index()

    summary_ext = df_ext.groupby(group_keys, dropna=False).agg(
        延長時間_td=('延長時間_td', 'sum'),
        延長売上=('売上', 'sum')
    ).reset_index()

    # 外部結合
    merged = pd.merge(summary_main, summary_ext, on=group_keys, how='outer')

    # 欠損値を補完
    merged['件数'] = merged['件数'].fillna(0).astype(int)
    merged['売上'] = (merged['通常売上'].fillna(0) + merged['延長売上'].fillna(0)).astype(int)
    total_time_td = merged['通常時間_td'].fillna(pd.Timedelta(0)) + merged['延長時間_td'].fillna(pd.Timedelta(0))

    # --- 4. 期間フィルタ（昨日以前） ---
    JST = timezone(timedelta(hours=+9), 'JST')
    today_in_japan = datetime.now(JST).date()
    yesterday_in_japan = today_in_japan - timedelta(days=1)
    merged = merged[merged['集計日'].dt.date <= yesterday_in_japan].copy()

    # --- 5. フォーマット変換 ---
    def timedelta_to_hhmm(td):
        if pd.isna(td) or td.total_seconds() < 0: return '0:00'
        ts = td.total_seconds()
        return f"{int(ts // 3600)}:{int((ts % 3600) // 60):02}"

    merged['月次タグ'] = merged['集計日'].dt.strftime('%Y年%m月')
    merged['日付'] = merged['集計日'].dt.strftime('%Y-%m-%d')
    merged['利用時間'] = total_time_td.apply(timedelta_to_hhmm)

    return merged[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上']]

def load_csv_with_encoding(uploaded_file_dict):
    """文字コードを自動判定して読み込む"""
    filename = next(iter(uploaded_file_dict))
    content = uploaded_file_dict[filename]
    try:
        return pd.read_csv(io.BytesIO(content), encoding='utf-8-sig')
    except UnicodeDecodeError:
        return pd.read_csv(io.BytesIO(content), encoding='cp932')

# --- メイン実行部分 ---
try:
    print("【手順1】タグ設置リストをアップロードしてください。")
    up_tag = files.upload()
    df_tag_master = load_csv_with_encoding(up_tag)
    df_mapping = df_tag_master[df_tag_master['媒体名'] == 'いいアプリ'].copy()
    df_mapping = df_mapping[['媒体側施設名', '媒体側設備名', '統一施設名', '統一設備名']].drop_duplicates()
    df_mapping = df_mapping.rename(columns={'媒体側施設名': 'f_key', '媒体側設備名': 'e_key', '統一施設名': '統一店名'})

    print("\n【手順2】「①設備予約（延長以外）.csv」をアップロードしてください。")
    up_main = files.upload()
    df_main_in = load_csv_with_encoding(up_main)

    print("\n【手順3】「②設備予約（延長のみ）.csv」をアップロードしてください。")
    up_ext = files.upload()
    df_ext_in = load_csv_with_encoding(up_ext)

    print("\n集計と照合を開始します...")
    res_df = perform_aggregation(df_main_in, df_ext_in)

    # 照合用キー作成
    res_df['f_tmp'] = res_df['施設名'].astype(str).str.strip()
    res_df['e_tmp'] = res_df['設備名'].astype(str).str.strip()
    df_mapping['f_tmp'] = df_mapping['f_key'].astype(str).str.strip()
    df_mapping['e_tmp'] = df_mapping['e_key'].astype(str).str.strip()

    # 結合
    merged_df = pd.merge(res_df, df_mapping[['f_tmp', 'e_tmp', '統一店名', '統一設備名']], on=['f_tmp', 'e_tmp'], how='left')
    unmatched = merged_df[merged_df['統一店名'].isna()][['施設名', '設備名']].drop_duplicates()

    merged_df['統一店名'] = merged_df['統一店名'].fillna('')
    merged_df['統一設備名'] = merged_df['統一設備名'].fillna('')

    out_name = 'いいアプリ利用実績_集計結果_統合版.csv'
    merged_df[['月次タグ', '日付', '施設名', 'プラン名', '設備名', '件数', '利用時間', '売上', '統一店名', '統一設備名']].to_csv(out_name, index=False, encoding='utf-8-sig')
    files.download(out_name)

    if not unmatched.empty:
        unmatched.to_csv('紐付け失敗リスト_いいアプリ.csv', index=False, encoding='utf-8-sig')
        files.download('紐付け失敗リスト_いいアプリ.csv')
    print("\n✅ 完了しました。")
except Exception as e:
    print(f"\n❌ エラー: {e}")

【手順1】タグ設置リストをアップロードしてください。


Saving 会議室シート_改 - タグ設置リスト (3).csv to 会議室シート_改 - タグ設置リスト (3).csv

【手順2】「①設備予約（延長以外）.csv」をアップロードしてください。


KeyboardInterrupt: 

In [ ]:
#@title ⑤全媒体データ統合ツール（日別集計・日付カラム版）

# ===================================================================
#               全媒体データ統合プログラム（日別・日付版）
# ===================================================================

import pandas as pd
from google.colab import files
import io
import unicodedata
import re

def merge_media_data_daily_fixed():
    print("各媒体の集計済みCSVファイル（日付カラムを含むもの）をすべて選択してアップロードしてください。")
    uploaded = files.upload()

    if not uploaded:
        print("ファイルが選択されませんでした。")
        return

    all_dfs = []

    # 媒体判定用のキーワードマップ
    media_map = {
        'インスタベース': 'インスタベース',
        'スペースマーケット': 'スペースマーケット',
        'スペイシー': 'スペイシー',
        'いいアプリ': 'いいアプリ',
        '会議室': 'いいアプリ'
    }

    # 時間（HH:MM）を分に変換するヘルパー関数
    def time_to_minutes(s):
        try:
            if pd.isna(s) or s == "": return 0
            parts = str(s).split(':')
            if len(parts) == 2:
                return int(parts[0]) * 60 + int(parts[1])
            return 0
        except:
            return 0

    # 分を時間（HH:MM）に変換するヘルパー関数
    def minutes_to_time(m):
        m = int(m)
        return f"{m // 60}:{m % 60:02}"

    for filename, content in uploaded.items():
        filename_norm = unicodedata.normalize('NFC', filename)

        # エラーファイルや失敗ファイルは除外
        if '失敗' in filename_norm or 'エラー' in filename_norm:
            continue

        media_name = "不明"
        for key, val in media_map.items():
            if key in filename_norm:
                media_name = val
                break

        print(f"-> 「{filename}」を読み込み中... [判定媒体: {media_name}]")

        try:
            # UTF-8で読み込み試行
            df = pd.read_csv(io.BytesIO(content), encoding='utf-8-sig')
        except:
            # 失敗した場合はShift-JIS(CP932)で読み込み
            df = pd.read_csv(io.BytesIO(content), encoding='cp932')

        df['媒体名'] = media_name

        # カラム名の統一（「利用日」を「日付」に変更、店舗名等のケア）
        if '日付' not in df.columns and '利用日' in df.columns:
            df = df.rename(columns={'利用日': '日付'})

        if '統一店名' not in df.columns and '統一施設名' in df.columns:
            df = df.rename(columns={'統一施設名': '統一店名'})

        # 必須データのクリーニング
        if '日付' not in df.columns:
            print(f"   ⚠️ 「{filename}」に「日付」カラムが見つからないため、このファイルはスキップします。")
            continue

        df['統一店名'] = df['統一店名'].astype(str).str.strip()
        df['統一設備名'] = df['統一設備名'].astype(str).str.strip()
        df['日付'] = df['日付'].astype(str).str.strip()

        # 不要なデータの除外（媒体不明、店名・設備名・日付が空のもの）
        df = df[
            (df['媒体名'] != "不明") &
            (df['統一店名'] != "") & (df['統一店名'] != "nan") &
            (df['統一設備名'] != "") & (df['統一設備名'] != "nan") &
            (df['日付'] != "") & (df['日付'] != "nan")
        ].copy()

        if not df.empty:
            # 時間を数値化（集計用）
            if '利用時間' in df.columns:
                df['利用分'] = df['利用時間'].apply(time_to_minutes)
            else:
                df['利用分'] = 0

            all_dfs.append(df)

    if not all_dfs:
        print("統合可能な有効データが見つかりませんでした。")
        return

    # 全データを結合
    combined_df = pd.concat(all_dfs, ignore_index=True)

    # --- 日別集計処理 ---
    # 日付、媒体名、統一店名、統一設備名でグループ化
    group_cols = ['日付', '媒体名', '統一店名', '統一設備名']

    # 件数、売上、利用分（分単位）を合計
    summary_df = combined_df.groupby(group_cols, as_index=False).agg({
        '件数': 'sum',
        '売上': 'sum',
        '利用分': 'sum'
    })

    # 分単位の合計を HH:MM 形式に戻す
    summary_df['利用時間'] = summary_df['利用分'].apply(minutes_to_time)

    # カラム順の整列
    final_cols = ['日付', '媒体名', '統一店名', '統一設備名', '件数', '利用時間', '売上']
    summary_df = summary_df[final_cols]

    # 日付として正しくソートするために一時変換
    try:
        # 様々な日付形式に対応するため、errors='coerce'で不正な形式をNaTに
        summary_df['temp_date'] = pd.to_datetime(summary_df['日付'], errors='coerce')
        summary_df = summary_df.sort_values(['temp_date', '媒体名', '統一店名'])
        summary_df = summary_df.drop(columns=['temp_date'])
    except:
        # 万が一変換に失敗した場合は文字列のままソート
        summary_df = summary_df.sort_values(['日付', '媒体名', '統一店名'])

    # CSV出力
    output_filename = '全媒体日別統合データ_最終集計.csv'
    summary_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
    files.download(output_filename)

    print(f"\n✅ 日別統合（日付カラム版）が完了しました！")
    print(f"「{output_filename}」をダウンロードしました。")
    print("-" * 30)
    print("【媒体別の日別データ数（行数）】")
    print(summary_df['媒体名'].value_counts())

# 実行
try:
    merge_media_data_daily_fixed()
except Exception as e:
    print(f"\n❌ エラーが発生しました: {e}")

各媒体の集計済みCSVファイル（日付カラムを含むもの）をすべて選択してアップロードしてください。


Saving 会議室シート_改 - スペイシー.csv to 会議室シート_改 - スペイシー.csv
Saving 会議室シート_改 - スペースマーケット.csv to 会議室シート_改 - スペースマーケット.csv
Saving 会議室シート_改 - インスタベース.csv to 会議室シート_改 - インスタベース.csv
Saving 会議室シート_改 - いいアプリ.csv to 会議室シート_改 - いいアプリ.csv
-> 「会議室シート_改 - スペイシー.csv」を読み込み中... [判定媒体: スペイシー]
-> 「会議室シート_改 - スペースマーケット.csv」を読み込み中... [判定媒体: スペースマーケット]
-> 「会議室シート_改 - インスタベース.csv」を読み込み中... [判定媒体: インスタベース]
-> 「会議室シート_改 - いいアプリ.csv」を読み込み中... [判定媒体: いいアプリ]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ 日別統合（日付カラム版）が完了しました！
「全媒体日別統合データ_最終集計.csv」をダウンロードしました。
------------------------------
【媒体別の日別データ数（行数）】
媒体名
インスタベース      18647
いいアプリ        17984
スペースマーケット    11621
スペイシー         2746
Name: count, dtype: int64


# **ここから下は龍﨑さん用のレポート作成ツール**

①[売上と経費シート](https://e-office.metabaseapp.com/public/dashboard/dcf06adb-c45a-499b-bb04-fa5e933ec826)から上の売上CSVデータを入手。期間絞って更新したい月分だけにしたほうが容量軽くて楽です。

②[分類分けシートダウンロード](https://docs.google.com/spreadsheets/d/1UNqPBUhHbpJHjz5Sx0KIETXCI1FETDcxnNwowe4L5KA/edit?gid=1010859939#gid=1010859939)

In [ ]:
# @title ⑥いいアプリ売上データ日別集計ツール（分類更新・設備予約追加版）

import pandas as pd
import numpy as np
import io
import unicodedata
from datetime import date
from google.colab import files

# --- 設定項目 ---
OUTPUT_FILE_NAME = 'daily_sales_summary_updated.csv'

def clean_and_convert_to_numeric(series):
    """金額の列をクリーンにして数値型に変換する関数（マイナス符号維持）"""
    cleaned_series = series.astype(str).str.replace(r'[^-\d.]', '', regex=True)
    cleaned_series = cleaned_series.replace('', '0')
    return pd.to_numeric(cleaned_series, errors='coerce').fillna(0)

def main():
    print("--- いいアプリ売上集計ツール (日別・新分類/設備予約対応版) ---")
    print("以下の2つのCSVファイルをアップロードしてください。")
    print("1. いいアプリの「売上データ」")
    print("2. 「売上分類分け (2).csv」")
    print("-" * 50)

    # 1. ファイルアップロード
    uploaded = files.upload()
    if not uploaded:
        print("ファイルが選択されませんでした。")
        return

    sales_df = None
    class_df = None

    # 2. ファイルの振り分け
    for filename, content in uploaded.items():
        filename_norm = unicodedata.normalize('NFC', filename)
        if '分類分け' in filename_norm:
            class_df = pd.read_csv(io.BytesIO(content))
            print(f"✅ 新分類マスタを確認: {filename}")
        else:
            try:
                sales_df = pd.read_csv(io.BytesIO(content), encoding='utf-8-sig')
            except:
                sales_df = pd.read_csv(io.BytesIO(content), encoding='cp932')
            print(f"✅ 売上データを読み込み: {filename}")

    if sales_df is None or class_df is None:
        print("\n❌ エラー: 「売上データ」と「売上分類分け.csv」の両方が必要です。")
        return

    # 3. 分類マスタの準備
    # 項目名 -> 分類 の辞書を作成
    class_map = dict(zip(class_df['項目名'].astype(str).str.strip(), class_df['分類'].astype(str).str.strip()))

    # 4. 売上データの集計処理
    df = sales_df.copy()
    df.columns = df.columns.str.strip()

    # 必須カラムの存在確認とクレンジング
    target_cols = ['利用プラン', '項目', '対象店舗', '発生日', '売上']
    for col in target_cols:
        if col not in df.columns:
            print(f"❌ エラー: 売上データに「{col}」カラムが見つかりません。")
            return
        df[col] = df[col].fillna('').astype(str).str.strip()

    # 日付変換
    df['日付_dt'] = pd.to_datetime(df['発生日'], errors='coerce')
    df.dropna(subset=['日付_dt'], inplace=True)
    df['日付'] = df['日付_dt'].dt.strftime('%Y-%m-%d')

    # 数値変換（マイナス対応）
    df['売上数値'] = clean_and_convert_to_numeric(df['売上'])

    # カテゴリ判定ロジック
    def classify(row):
        plan = row['利用プラン']
        item = row['項目']

        # 1. 項目名が「外部媒体売上」なら最優先で「外部媒体売上」に分類
        if item == '外部媒体売上':
            return '外部媒体売上'

        # 2. マスタから分類を取得、なければ「その他」
        return class_map.get(plan, 'その他')

    df['カテゴリ'] = df.apply(classify, axis=1)

    # 5. 集計（ピボット）
    summary = df.groupby(['日付', '対象店舗', 'カテゴリ'])['売上数値'].sum().reset_index()
    pivot_summary = summary.pivot_table(
        index=['日付', '対象店舗'],
        columns='カテゴリ',
        values='売上数値',
        fill_value=0
    ).reset_index()

    pivot_summary.rename(columns={'対象店舗': '店名'}, inplace=True)

    # 指定された出力カラム構成
    # 日付、店名、売上合計、従量課金、月額契約、分配金、その他、設備予約、外部媒体売上
    required_cats = ['従量課金', '月額契約', '分配金', 'その他', '設備予約', '外部媒体売上']
    for cat in required_cats:
        if cat not in pivot_summary.columns:
            pivot_summary[cat] = 0

    # 売上合計を計算
    pivot_summary['売上合計'] = pivot_summary[required_cats].sum(axis=1)

    # 6. 日付の補完（データがない日を0埋め）
    all_stores = sorted(list(pivot_summary['店名'].unique()))
    all_dates_dt = pd.to_datetime(pivot_summary['日付'])
    continuous_dates = pd.date_range(start=all_dates_dt.min(), end=all_dates_dt.max(), freq='D').strftime('%Y-%m-%d')

    all_combinations = pd.MultiIndex.from_product(
        [continuous_dates, all_stores],
        names=['日付', '店名']
    ).to_frame(index=False)

    final_df = pd.merge(all_combinations, pivot_summary, on=['日付', '店名'], how='left').fillna(0)

    # 指定された順番にカラムを整理
    final_cols = ['日付', '店名', '売上合計', '従量課金', '月額契約', '分配金', 'その他', '設備予約', '外部媒体売上']
    final_df = final_df[final_cols]
    final_df = final_df.sort_values(by=['日付', '店名'])

    # 7. 保存とダウンロード
    final_df.to_csv(OUTPUT_FILE_NAME, index=False, encoding='utf-8-sig')
    print(f"\n✨ 集計が完了しました！")
    print(f"分類分けできない項目は自動的に「その他」へ集計されています。")
    print(f"出力ファイル: {OUTPUT_FILE_NAME}")

    files.download(OUTPUT_FILE_NAME)

if __name__ == '__main__':
    try:
        main()
    except Exception as e:
        print(f"\n❌ 予期せぬエラーが発生しました: {e}")

--- いいアプリ売上集計ツール (日別・新分類/設備予約対応版) ---
以下の2つのCSVファイルをアップロードしてください。
1. いいアプリの「売上データ」
2. 「売上分類分け (2).csv」
--------------------------------------------------


ファイルが選択されませんでした。


[ここに貼り付け](https://docs.google.com/spreadsheets/d/1UNqPBUhHbpJHjz5Sx0KIETXCI1FETDcxnNwowe4L5KA/edit?usp=sharing)

⑦に必要なデータ

 1. 会議室シート_改 - 全体元データ.csv
 2. 会議室シート_改 - グラフの順番.csv
 3. 会議室シート_改 - タグ設置リスト.csv
 4. いいアプリ売上データ置き場 - 連携.csv

In [ ]:
# @title ⑦グラフ付きのレポート生成ツール（前月データ補完強化版・⑧売上進捗統合）
# ==========================================
# 1. システム準備
# ==========================================
print("--- [1/5] システム準備中 ---")
!pip install plotly -q

import pandas as pd
import json
import io
import unicodedata
import datetime
from google.colab import files
import sys
import re
import os

# 数値クレンジング（円、カンマ除去）
def clean_num(series):
    return pd.to_numeric(series.astype(str).str.replace('円', '').str.replace(',', ''), errors='coerce').fillna(0)

# 【重要】文字コードの揺れ（濁点問題）を完全に解消する関数
def fix_text(text):
    if pd.isna(text): return ""
    # NFC形式に統一、スペース除去、全角を半角へ
    t = unicodedata.normalize('NFC', str(text)).strip().replace(" ", "").replace("　", "")
    return t

# 媒体名を確実に判定する関数
def get_media_name(m):
    m = fix_text(m)
    if "いいアプリ" in m or "会議室" in m: return "いいアプリ"
    if "インスタ" in m: return "インスタベース"
    if "スペイシー" in m: return "スペイシー"
    if "スペースマーケット" in m or "スペマ" in m: return "スペースマーケット"
    return m

# ==========================================
# 2. ファイルアップロード
# ==========================================
print("\n" + "="*60)
print(" 【重要】以下の4つのCSVファイルをアップロードしてください。")
print(" 1. 会議室シート_改 - 全体元データ.csv")
print(" 2. 会議室シート_改 - グラフの順番.csv")
print(" 3. 会議室シート_改 - タグ設置リスト.csv")
print(" 4. いいアプリ売上データ置き場 - 連携.csv")
print("="*60 + "\n")

uploaded = files.upload()

def find_file(keyword):
    # ファイル名自体も正規化して検索
    for name in uploaded.keys():
        if keyword in unicodedata.normalize('NFC', name): return name
    return None

data_f = find_file("全体元データ")
order_f = find_file("グラフの順番")
tags_f = find_file("タグ設置リスト")
store_f = find_file("連携")

if not all([data_f, order_f, tags_f, store_f]):
    print(f"\n【エラー】ファイルが見つかりません。 (検出状況: 元データ={data_f}, 順番={order_f}, タグ={tags_f}, 連携={store_f})")
    sys.exit()

# ==========================================
# 3. データの集計・同期・補完
# ==========================================
def load_csv(name):
    for enc in ['utf-8-sig', 'shift-jis', 'cp932', 'utf-8']:
        try: return pd.read_csv(io.BytesIO(uploaded[name]), encoding=enc)
        except: continue
    return pd.read_csv(io.BytesIO(uploaded[name]))

df_main = load_csv(data_f)
df_order = load_csv(order_f)
df_tags = load_csv(tags_f)
df_store = load_csv(store_f)

# --- 全データの超強力正規化 ---
print("-> データの文字コードを最適化中...")
df_main['統一店名'] = df_main['統一店名'].apply(fix_text)
df_main['統一設備名'] = df_main['統一設備名'].apply(fix_text)
df_main['媒体名'] = df_main['媒体名'].apply(get_media_name)

df_order.iloc[:, 0] = df_order.iloc[:, 0].apply(fix_text)
df_tags['統一施設名'] = df_tags['統一施設名'].apply(fix_text)
df_tags['媒体側施設名'] = df_tags['媒体側施設名'].apply(fix_text)

df_store['店名'] = df_store['店名'].apply(fix_text)

# 日付処理
df_main['dt'] = pd.to_datetime(df_main['日付'].astype(str).str.replace('/', '-'), errors='coerce')
df_main['月次タグ'] = df_main['dt'].dt.strftime('%Y年%m月')
cutoff_dt = df_main['dt'].max()
current_month_label = cutoff_dt.strftime('%Y年%m月')

# 連携データの日付同期
df_store['dt'] = pd.to_datetime(df_store['日付'], errors='coerce')
df_store = df_store[df_store['dt'] <= cutoff_dt].copy()
df_store['月次'] = df_store['dt'].dt.strftime('%Y年%m月')

# 名寄せ（連携データの店名を統一施設名に変換）
mapping = dict(zip(df_tags['媒体側施設名'], df_tags['統一施設名']))
df_store['統一店名'] = df_store['店名'].map(mapping).fillna(df_store['店名'])

print(f"-> 基準日: {cutoff_dt.strftime('%Y/%m/%d')} (当月: {current_month_label})")

store_cols = ['月額契約', '分配金', 'その他', '従量課金', '設備予約', '外部媒体売上']
for c in store_cols: df_store[c] = clean_num(df_store[c])
df_store_monthly = df_store.groupby(['月次', '統一店名'])[store_cols].sum().reset_index()

# カレンダー作成 (13ヶ月分)
months_13_dt = [(cutoff_dt.replace(day=1) - pd.DateOffset(months=i)) for i in range(12, -1, -1)]
month_labels = [d.strftime('%Y年%m月') for d in months_13_dt]
# 前月ラベルの取得
previous_month_label = month_labels[-2] if len(month_labels) >= 2 else None

def calc_pct(curr, prev):
    if prev == 0: return "-" if curr == 0 else "新規"
    return f"{int((curr / prev) * 100)}%"

df_main['売上'] = clean_num(df_main['売上'])
df_main['件数'] = clean_num(df_main['件数'])
all_media = ['いいアプリ', 'インスタベース', 'スペイシー', 'スペースマーケット']
ordered_stores = df_order.iloc[:, 0].dropna().unique().tolist()

# ------------------------------------------
# データ抽出コアエンジン
# ------------------------------------------
def get_data(s_name=None, f_name=None, mode="linkage", is_global=False):
    res_list = []
    for label in month_labels:
        row = {"month": label}
        if mode == "original":
            # 元データ参照（会議室予約）
            if is_global:
                m_data = df_main[df_main['月次タグ']==label]
            elif f_name:
                m_data = df_main[(df_main['統一店名']==s_name) & (df_main['統一設備名']==f_name) & (df_main['月次タグ']==label)]
            else:
                m_data = df_main[(df_main['統一店名']==s_name) & (df_main['月次タグ']==label)]

            if f_name:
                row.update({"total_sales": int(m_data['売上'].sum()), "total_counts": int(m_data['件数'].sum())})
                for m in all_media:
                    row[f"s_{m}"] = int(m_data[m_data['媒体名']==m]['売上'].sum())
                    row[f"c_{m}"] = int(m_data[m_data['媒体名']==m]['件数'].sum())
            else:
                for m in all_media: row[m] = int(m_data[m_data['媒体名']==m]['売上'].sum())
                row["total_sales"] = sum([row[m] for m in all_media])
        else:
            # 連携データ参照（店舗全体実績：当月・前月自動補完ロジック）
            if is_global:
                g_df = df_store_monthly[df_store_monthly['月次']==label]
                base = g_df[store_cols].sum().to_dict()

                # 最新月の補完 (いいアプリ予約・外部予約を元データから常に反映)
                if label == current_month_label:
                    m_data = df_main[df_main['月次タグ']==label]
                    base['設備予約'] = int(m_data[m_data['媒体名']=='いいアプリ']['売上'].sum())
                    base['外部媒体売上'] = int(m_data[m_data['媒体名']!='いいアプリ']['売上'].sum())

                # 前月分の外部媒体売上が0の場合の補完
                elif label == previous_month_label and base.get('外部媒体売上', 0) == 0:
                    m_data = df_main[df_main['月次タグ']==label]
                    base['外部媒体売上'] = int(m_data[m_data['媒体名']!='いいアプリ']['売上'].sum())

                row.update({c: int(base.get(c, 0)) for c in store_cols})
            else:
                s_df = df_store_monthly[(df_store_monthly['統一店名']==s_name) & (df_store_monthly['月次']==label)]
                base = s_df.iloc[0].to_dict() if not s_df.empty else {c: 0 for c in store_cols}

                # 最新月の補完
                if label == current_month_label:
                    m_data = df_main[(df_main['統一店名']==s_name) & (df_main['月次タグ']==label)]
                    base['設備予約'] = int(m_data[m_data['媒体名']=='いいアプリ']['売上'].sum())
                    base['外部媒体売上'] = int(m_data[m_data['媒体名']!='いいアプリ']['売上'].sum())

                # 前月分の外部媒体売上が0の場合の補完
                elif label == previous_month_label and base.get('外部媒体売上', 0) == 0:
                    m_data = df_main[(df_main['統一店名']==s_name) & (df_main['月次タグ']==label)]
                    base['外部媒体売上'] = int(m_data[m_data['媒体名']!='いいアプリ']['売上'].sum())

                row.update({c: int(base[c]) for c in store_cols})
            row["total_sales"] = sum([row[c] for c in store_cols])
        res_list.append(row)

    # 前月比
    mom_cols = ['従量課金', '設備予約', '外部媒体売上'] if mode=="linkage" else all_media
    for i in range(len(res_list)):
        curr, prev = res_list[i], (res_list[i-1] if i > 0 else None)
        if mode == "original": curr["total_sales_mom"] = calc_pct(curr["total_sales"], prev["total_sales"] if prev else 0)
        for c in mom_cols:
            k = f"s_{c}" if f_name else c
            curr[f"{k}_mom"] = calc_pct(curr[k], prev[k] if prev else 0)
    return res_list

# ------------------------------------------
# 5日刻み売上進捗データの生成 (⑧統合)
# ------------------------------------------
print("-> 5日刻み売上進捗データを生成中...")

# いいアプリ(内部)と外部媒体を日別・店舗別に集計
_is_internal = df_main['媒体名'] == 'いいアプリ'
daily_room = df_main.groupby(['dt', '統一店名', _is_internal.rename('is_int')])['売上'].sum().unstack(fill_value=0).reset_index()
if True not in daily_room.columns: daily_room[True] = 0
if False not in daily_room.columns: daily_room[False] = 0
daily_room.rename(columns={True: '設備予約_内部', False: '外部媒体売上_部屋'}, inplace=True)

# 連携データからベース売上(月額契約・分配金・その他・従量課金)を日別集計
base_cols_prog = ['従量課金', '月額契約', '分配金', 'その他']
df_store_prog = df_store.copy()
for c in base_cols_prog:
    if c not in df_store_prog.columns: df_store_prog[c] = 0
df_store_prog['ベース売上'] = df_store_prog[base_cols_prog].sum(axis=1)
daily_base = df_store_prog.groupby(['dt', '統一店名'])['ベース売上'].sum().reset_index()

# 結合して当月データに限定
df_prog = pd.merge(daily_base, daily_room, on=['dt', '統一店名'], how='outer').fillna(0)
current_ym = cutoff_dt.strftime('%Y-%m')
df_prog_cur = df_prog[df_prog['dt'].dt.strftime('%Y-%m') == current_ym].copy()

# マイルストーン日付(5日, 10日, 15日, 20日, 25日, 月末 + 最新日)
month_start = cutoff_dt.replace(day=1)
full_dates = pd.date_range(month_start, cutoff_dt, freq='D')
milestone_dates = sorted(set(
    [d for d in full_dates if d.day in [5, 10, 15, 20, 25] or d.is_month_end]
    + [cutoff_dt]
))

# 全店合計の進捗
all_daily_p = df_prog_cur.groupby('dt').agg(
    {'ベース売上':'sum', '設備予約_内部':'sum', '外部媒体売上_部屋':'sum'}
).reindex(full_dates, fill_value=0)
all_cum_st = (all_daily_p['ベース売上'] + all_daily_p['設備予約_内部']).cumsum()
all_cum_ex = all_daily_p['外部媒体売上_部屋'].cumsum()
progress_all_store = [int(all_cum_st.loc[d]) for d in milestone_dates]
progress_all_ext = [int(all_cum_ex.loc[d]) for d in milestone_dates]

# 店舗別の進捗
progress_stores = []
for sname in ordered_stores:
    s_data = df_prog_cur[df_prog_cur['統一店名'] == sname]
    s_agg = s_data.groupby('dt').agg(
        {'ベース売上':'sum', '設備予約_内部':'sum', '外部媒体売上_部屋':'sum'}
    ).reindex(full_dates, fill_value=0)
    s_cum_st = (s_agg['ベース売上'] + s_agg['設備予約_内部']).cumsum()
    s_cum_ex = s_agg['外部媒体売上_部屋'].cumsum()
    progress_stores.append({
        'name': sname,
        'store_total': [int(s_cum_st.loc[d]) for d in milestone_dates],
        'external': [int(s_cum_ex.loc[d]) for d in milestone_dates]
    })
progress_milestones = [d.strftime('%m/%d') for d in milestone_dates]

# ==========================================
# 4. JSON構築
# ==========================================
print("\n--- [4/5] ダッシュボードを構築中 ---")
js_master = {
    "g_all": get_data(is_global=True, mode="linkage"),
    "g_room": get_data(is_global=True, mode="original"),
    "stores": []
}
for idx, sname in enumerate(ordered_stores):
    s_main_rows = df_main[df_main['統一店名'] == sname]
    sorted_facs = s_main_rows.groupby('統一設備名')['売上'].sum().sort_values(ascending=False).index.tolist()
    fac_data = [{"name": f, "data": get_data(sname, f, mode="original")} for f in sorted_facs]
    js_master["stores"].append({
        "id": f"st_{idx}", "name": sname,
        "summary": get_data(sname, mode="linkage"),
        "original_summary": get_data(sname, mode="original"),
        "facilities": fac_data
    })
js_master["progress"] = {
    "month": current_month_label,
    "milestones": progress_milestones,
    "all": {"store_total": progress_all_store, "external": progress_all_ext},
    "stores": progress_stores
}

# ==========================================
# 5. HTML生成
# ==========================================
json_data_str = json.dumps(js_master, ensure_ascii=False)
media_list_str = json.dumps(all_media, ensure_ascii=False)

html_template = """
<!DOCTYPE html><html><head><meta charset="UTF-8"><title>稼働実績ダッシュボード</title>
<script src="https://cdn.plot.ly/plotly-2.24.1.min.js"></script>
<style>
:root { --primary: #22923f; --primary-light: #e8f8ed; --bg: #f8f9fa; --up: #e74c3c; --down: #2980b9; }
body { font-family: sans-serif; margin: 0; display: flex; background: var(--bg); }
nav { width: 260px; background: #2f3640; height: 100vh; position: sticky; top: 0; overflow-y: auto; color: white; z-index: 1000; flex-shrink: 0; }
nav h3 { padding: 20px; background: #222f3e; margin: 0; font-size: 1rem; }
nav ul { list-style: none; padding: 0; }
nav li a { display: block; padding: 12px 20px; color: #ced6e0; text-decoration: none; border-bottom: 1px solid #34495e; font-size: 0.82rem; }
nav li a:hover { background: var(--primary); color: white; }
.title-bar { position: sticky; top: 0; background: white; padding: 10px 25px; border-bottom: 3px solid var(--primary); box-shadow: 0 2px 5px rgba(0,0,0,0.08); z-index: 900; display: flex; justify-content: space-between; align-items: center; }
.title-bar h2 { margin: 0; font-size: 1.2rem; color: var(--primary); }
main { flex: 1; min-width: 0; }
.section { padding: 10px 25px 80px; scroll-margin-top: 10px; }
.card { background: white; border-radius: 12px; box-shadow: 0 4px 15px rgba(0,0,0,0.05); margin-bottom: 35px; border: 1px solid #dfe6e9; overflow: hidden; }
.card-h { background: #f8f9fa; padding: 12px 20px; font-weight: bold; border-bottom: 1px solid #dfe6e9; }
.card-b { padding: 20px; }
.table-c { overflow-x: auto; border-radius: 8px; border: 1px solid #dfe6e9; margin-top: 15px; }
table { width: 100%; border-collapse: collapse; font-size: 0.74rem; background: white; }
th, td { border: 1px solid #eee; padding: 6px; text-align: right; }
th { background: #f1f2f6; text-align: center; white-space: nowrap; }
td:first-child { text-align: center; font-weight: bold; background: #fafafa; position: sticky; left: 0; }
.mom { font-weight: bold; font-size: 0.7rem; }
.mom.up { color: var(--up); }
.mom.down { color: var(--down); }
.tot { background: var(--primary-light); font-weight: bold; }
</style></head><body>
<nav><h3>実績メニュー</h3><ul id="nav-ul"></ul></nav>
<main id="main-c"></main>
<script>
const data = """ + json_data_str + """;
const media = """ + media_list_str + """;
const content = document.getElementById('main-c');
const nav = document.getElementById('nav-ul');

let nHtml = '<li><a href="#g-all">全店舗合計 (総合)</a></li><li><a href="#g-room">全店舗合計 (会議室のみ)</a></li><li><a href="#progress" style="color:#f1c40f;font-weight:bold">5日刻み売上進捗</a></li>';
data.stores.forEach(s => { nHtml += `<li><a href="#${s.id}">${s.name}</a></li>`; });
nav.innerHTML = nHtml;

function getMomClass(m) { if(!m || m==="-" || m==="新規") return ""; const v = parseInt(m.replace('%','')); return v > 100 ? "up" : (v < 100 ? "down" : ""); }

function getStoreTbl(rows) {
    const cats = ['月額契約', '分配金', 'その他', '従量課金', '設備予約', '外部媒体売上'];
    const momCats = ['従量課金', '設備予約', '外部媒体売上'];
    let h = '<div class="table-c"><table><thead><tr><th>月次</th><th style="background:#e8f8ed">合計売上</th>';
    cats.forEach(c => { h += (momCats.includes(c)) ? `<th colspan="2">${c}</th>` : `<th>${c}</th>`; });
    h += '</tr></thead><tbody>';
    rows.forEach(r => {
        h += `<tr><td>${r.month}</td><td class="tot">${r.total_sales.toLocaleString()}</td>`;
        cats.forEach(c => {
            h += `<td>${(r[c]||0).toLocaleString()}</td>`;
            if(momCats.includes(c)) { let m = r[c+"_mom"] || "-"; h += `<td class="mom ${getMomClass(m)}">${m}</td>`; }
        });
        h += '</tr>';
    });
    return h + '</tbody></table></div>';
}

function getOriginalTbl(rows, cats) {
    let h = '<div class="table-c"><table><thead><tr><th>月次</th><th style="background:#e8f8ed">合計売上</th><th style="background:#e8f8ed">前月比</th>';
    cats.forEach(c => h += `<th>${c}</th>`);
    h += '</tr></thead><tbody>';
    rows.forEach(r => {
        h += `<tr><td>${r.month}</td><td class="tot">${r.total_sales.toLocaleString()}</td><td class="mom ${getMomClass(r.total_sales_mom)}">${r.total_sales_mom}</td>`;
        cats.forEach(c => h += `<td>${(r[c]||0).toLocaleString()}</td>`);
        h += '</tr>';
    });
    return h + '</tbody></table></div>';
}

function getFacTbl(rows) {
    let h = '<div class="table-c"><table><thead><tr><th rowspan="2">月次</th><th colspan="3" style="background:#e8f8ed">施設全体</th>';
    media.forEach(m => h += `<th colspan="3">${m}</th>`);
    h += '</tr><tr><th style="background:#f1f2f6">売上</th><th style="background:#f1f2f6">前月比</th><th style="background:#f1f2f6">件数</th>';
    media.forEach(m => h += `<th>売上</th><th>前月比</th><th>件数</th>`);
    h += '</tr></thead><tbody>';
    rows.forEach(r => {
        h += `<tr><td>${r.month}</td><td class="tot">${r.total_sales.toLocaleString()}</td><td class="mom ${getMomClass(r.total_sales_mom)}">${r.total_sales_mom}</td><td>${r.total_counts.toLocaleString()}</td>`;
        media.forEach(m => {
            let m_mom = r['s_'+m+'_mom'] || "-";
            h += `<td>${(r['s_'+m]||0).toLocaleString()}</td><td class="mom ${getMomClass(m_mom)}">${m_mom}</td><td>${(r['c_'+m]||0).toLocaleString()}</td>`;
        });
        h += '</tr>';
    });
    return h + '</tbody></table></div>';
}

let sHtml = `<section id="g-all" class="section"><div class="title-bar"><h2>📊 全店舗合計 (総合)</h2><span>基準日: """ + cutoff_dt.strftime('%Y/%m/%d') + """</span></div><div class="card"><div class="card-h">全社実績 (連携データベース)</div><div class="card-b"><div id="c-gall" style="height:400px;"></div><div id="t-gall"></div></div></div></section>
<section id="g-room" class="section"><div class="title-bar"><h2>💼 全店舗合計 (会議室のみ)</h2></div><div class="card"><div class="card-h">全社 会議室予約内訳 (全体元データベース)</div><div class="card-b"><div id="c-groom" style="height:400px;"></div><div id="t-groom"></div></div></div></section>
<section id="progress" class="section"><div class="title-bar"><h2>📈 ${data.progress.month} 5日刻み売上進捗</h2><span>基準日: """ + cutoff_dt.strftime('%Y/%m/%d') + """</span></div>
<div class="card"><div class="card-h">全店舗 売上進捗推移（累計）</div><div class="card-b"><div id="c-prog-line" style="height:450px;"></div></div></div>
<div class="card"><div class="card-h">店舗別 最新売上内訳</div><div class="card-b"><div id="c-prog-bar" style="height:400px;"></div></div></div>
<div class="card"><div class="card-h">5日刻み売上進捗テーブル</div><div class="card-b"><div id="t-prog"></div></div></div>
</section>`;

data.stores.forEach(s => {
    sHtml += `<section id="${s.id}" class="section"><div class="title-bar"><h2>🏠 ${s.name}</h2><span>基準日: """ + cutoff_dt.strftime('%Y/%m/%d') + """</span></div>
        <div class="card"><div class="card-h">① 店舗全体実績 (連携データベース)</div><div class="card-b"><div id="c-s-${s.id}" style="height:400px;"></div><div id="t-s-${s.id}"></div></div></div>
        <div class="card"><div class="card-h">② 会議室予約合計 (全体元データベース)</div><div class="card-b"><div id="c-sori-${s.id}" style="height:400px;"></div><div id="t-sori-${s.id}"></div></div></div>`;
    s.facilities.forEach((f, i) => {
        sHtml += `<div class="card"><div class="card-h">【設備】${f.name} (全体元データ参照)</div><div class="card-b"><div style="display:flex; flex-wrap:wrap; gap:15px;"><div id="c-fs-${s.id}-${i}" style="flex:1; min-width:320px; height:380px;"></div><div id="c-fc-${s.id}-${i}" style="flex:1; min-width:320px; height:380px;"></div></div><div id="t-f-${s.id}-${i}"></div></div></div>`;
    });
    sHtml += '</section>';
});
content.innerHTML = sHtml;

const rendered = new Set();
function render(id) {
    if(rendered.has(id)) return;
    const cfg = { responsive:true, displayModeBar:false };
    const lay = { barmode:'stack', margin:{t:40,b:40,l:60,r:15}, font:{size:11}, legend:{orientation:'h',y:-0.15}, colorway:['#22923f','#3498db','#9b59b6','#f1c40f','#e67e22','#e74c3c'] };
    const s_cats = ['月額契約', '分配金', 'その他', '従量課金', '設備予約', '外部媒体売上'];

    if(id==='g-all'){
        Plotly.newPlot('c-gall', s_cats.map(c=>({x:data.g_all.map(r=>r.month),y:data.g_all.map(r=>r[c]),name:c,type:'bar'})).filter(t=>t.y.some(v=>v>0)), lay, cfg);
        document.getElementById('t-gall').innerHTML = getStoreTbl(data.g_all);
    } else if(id==='g-room'){
        Plotly.newPlot('c-groom', media.map(c=>({x:data.g_room.map(r=>r.month),y:data.g_room.map(r=>r[c]),name:c,type:'bar'})).filter(t=>t.y.some(v=>v>0)), lay, cfg);
        document.getElementById('t-groom').innerHTML = getOriginalTbl(data.g_room, media);
    } else if(id==='progress'){
        const pg = data.progress;
        const ms = pg.milestones;
        const pLay = { margin:{t:50,b:60,l:80,r:15}, font:{size:11}, legend:{orientation:'h',y:-0.2}, colorway:['#2c3e50','#22923f','#3498db','#9b59b6','#f1c40f','#e67e22','#e74c3c','#1abc9c','#e91e63','#607d8b','#795548','#00bcd4','#ff5722','#4caf50','#673ab7','#ff9800','#009688','#f44336','#3f51b5','#cddc39'] };
        // 折れ線グラフ：店舗別の月間売上累計推移
        const lineTraces = [{x:ms,y:pg.all.store_total.map((v,i)=>v+pg.all.external[i]),name:'全店合計',type:'scatter',mode:'lines+markers',line:{width:3,dash:'dash',color:'#000'}}];
        pg.stores.forEach(s=>{lineTraces.push({x:ms,y:s.store_total.map((v,i)=>v+s.external[i]),name:s.name,type:'scatter',mode:'lines+markers'});});
        Plotly.newPlot('c-prog-line',lineTraces,{...pLay,title:pg.month+' 月間売上累計推移（5日刻み）',xaxis:{title:'日付'},yaxis:{title:'累計売上（円）'}},cfg);
        // 積み上げ棒グラフ：最新時点の店舗別内訳
        const li = ms.length-1;
        const sNames = pg.stores.map(s=>s.name);
        Plotly.newPlot('c-prog-bar',[
            {x:sNames,y:pg.stores.map(s=>s.store_total[li]),name:'店舗全体売上',type:'bar',marker:{color:'#22923f'}},
            {x:sNames,y:pg.stores.map(s=>s.external[li]),name:'外部媒体売上',type:'bar',marker:{color:'#3498db'}}
        ],{...pLay,barmode:'stack',title:'最新時点 ('+ms[li]+') 店舗別売上内訳'},cfg);
        // テーブル
        let tbl='<div class="table-c"><table><thead><tr><th rowspan="2" style="min-width:120px;position:sticky;left:0;background:#f1f2f6;z-index:1">店舗名</th>';
        ms.forEach(m=>{tbl+=`<th colspan="3" style="background:#e8f8ed">~${m}</th>`;});
        tbl+='</tr><tr>';
        ms.forEach(()=>{tbl+='<th>店舗全体</th><th>外部媒体</th><th style="background:#ffeaa7">合計</th>';});
        tbl+='</tr></thead><tbody>';
        tbl+='<tr style="background:#e8f8ed;font-weight:bold"><td style="position:sticky;left:0;background:#e8f8ed;z-index:1">全店合計</td>';
        ms.forEach((_,i)=>{const st=pg.all.store_total[i],ex=pg.all.external[i];tbl+=`<td>${st.toLocaleString()}</td><td>${ex.toLocaleString()}</td><td style="background:#ffeaa7;font-weight:bold">${(st+ex).toLocaleString()}</td>`;});
        tbl+='</tr>';
        pg.stores.forEach(s=>{
            tbl+=`<tr><td style="position:sticky;left:0;background:#fafafa;z-index:1">${s.name}</td>`;
            ms.forEach((_,i)=>{const st=s.store_total[i],ex=s.external[i];tbl+=`<td>${st.toLocaleString()}</td><td>${ex.toLocaleString()}</td><td style="background:#ffeaa7">${(st+ex).toLocaleString()}</td>`;});
            tbl+='</tr>';
        });
        tbl+='</tbody></table></div>';
        document.getElementById('t-prog').innerHTML=tbl;
    } else {
        const s = data.stores.find(x => x.id === id); if(!s) return;
        Plotly.newPlot('c-s-'+id, s_cats.map(c=>({x:s.summary.map(r=>r.month),y:s.summary.map(r=>r[c]),name:c,type:'bar'})).filter(t=>t.y.some(v=>v>0)), lay, cfg);
        document.getElementById('t-s-'+id).innerHTML = getStoreTbl(s.summary);
        Plotly.newPlot('c-sori-'+id, media.map(c=>({x:s.original_summary.map(r=>r.month),y:s.original_summary.map(r=>r[c]),name:c,type:'bar'})).filter(t=>t.y.some(v=>v>0)), lay, cfg);
        document.getElementById('t-sori-'+id).innerHTML = getOriginalTbl(s.original_summary, media);
        s.facilities.forEach((f,i)=>{
            Plotly.newPlot(`c-fs-${id}-${i}`, media.map(m=>({x:f.data.map(r=>r.month),y:f.data.map(r=>r["s_"+m]),name:m,type:'bar'})).filter(t=>t.y.some(v=>v>0)), {...lay, title:'売上推移'}, cfg);
            Plotly.newPlot(`c-fc-${id}-${i}`, media.map(m=>({x:f.data.map(r=>r.month),y:f.data.map(r=>r["c_"+m]),name:m,type:'bar'})).filter(t=>t.y.some(v=>v>0)), {...lay, title:'件数推移'}, cfg);
            document.getElementById(`t-f-${id}-${i}`).innerHTML = getFacTbl(f.data);
        });
    }
    rendered.add(id);
}
const obs = new IntersectionObserver((es)=>{ es.forEach(e=>{ if(e.isIntersecting) render(e.target.id); }); }, {threshold:0.05});
document.querySelectorAll('.section').forEach(s=>obs.observe(s));
</script></body></html>
"""

filename = f"稼働実績ダッシュボード_{cutoff_dt.strftime('%Y%m%d')}.html"
with open(filename, "w", encoding="utf-8") as f: f.write(html_template)
print(f"\n--- [5/5] 完了！ ---")
files.download(filename)

--- [1/5] システム準備中 ---

 【重要】以下の4つのCSVファイルをアップロードしてください。
 1. 会議室シート_改 - 全体元データ.csv
 2. 会議室シート_改 - グラフの順番.csv
 3. 会議室シート_改 - タグ設置リスト.csv
 4. いいアプリ売上データ置き場 - 連携.csv



Saving 会議室シート_改 - タグ設置リスト (3).csv to 会議室シート_改 - タグ設置リスト (3).csv
Saving 会議室シート_改 - グラフの順番 (2).csv to 会議室シート_改 - グラフの順番 (2).csv
Saving 会議室シート_改 - 全体元データ (2).csv to 会議室シート_改 - 全体元データ (2).csv
Saving いいアプリ売上データ置き場 - 連携 (2).csv to いいアプリ売上データ置き場 - 連携 (2).csv
-> データの文字コードを最適化中...
-> 基準日: 2026/02/02 (当月: 2026年02月)

--- [4/5] ダッシュボードを構築中 ---

--- [5/5] 完了！ ---


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# @title ⑧ 5日刻み・店舗別売上進捗集計ツール

import pandas as pd
import io
import unicodedata
from google.colab import files
from datetime import datetime

def clean_num(series):
    return pd.to_numeric(series.astype(str).str.replace(r'[^-\d.]', '', regex=True), errors='coerce').fillna(0)

def fix_text(text):
    if pd.isna(text): return ""
    return unicodedata.normalize('NFC', str(text)).strip().replace(" ", "").replace("　", "")

def main_progress_tool_v2():
    print("以下の3つのCSVファイルをアップロードしてください。")
    print("1. 会議室シート_改 - 全体元データ.csv")
    print("2. 会議室シート_改 - タグ設置リスト.csv")
    print("3. いいアプリ売上データ置き場 - 連携.csv")
    print("-" * 50)

    uploaded = files.upload()
    if not uploaded:
        print("ファイルが選択されませんでした。")
        return

    def find_file(keyword):
        for name in uploaded.keys():
            if keyword in unicodedata.normalize('NFC', name): return name
        return None

    data_f = find_file("全体元データ")
    tags_f = find_file("タグ設置リスト")
    store_f = find_file("連携")

    if not all([data_f, tags_f, store_f]):
        print("❌ 必要なファイルが見つかりません。アップロードしたファイル名を確認してください。")
        return

    # 読み込み
    def load_csv(name):
        try: return pd.read_csv(io.BytesIO(uploaded[name]), encoding='utf-8-sig')
        except: return pd.read_csv(io.BytesIO(uploaded[name]), encoding='cp932')

    df_room = load_csv(data_f)
    df_tags = load_csv(tags_f)
    df_link = load_csv(store_f)

    # 文字正規化（不一致防止）
    df_room['統一店名'] = df_room['統一店名'].apply(fix_text)
    df_room['媒体名'] = df_room['媒体名'].apply(fix_text)
    df_tags['媒体側施設名'] = df_tags['媒体側施設名'].apply(fix_text)
    df_tags['統一施設名'] = df_tags['統一施設名'].apply(fix_text)
    df_link['店名'] = df_link['店名'].apply(fix_text)

    # 日付と数値の処理
    df_room['dt'] = pd.to_datetime(df_room['日付'].astype(str).str.replace('/', '-'), errors='coerce')
    df_link['dt'] = pd.to_datetime(df_link['日付'], errors='coerce')
    df_room['売上'] = clean_num(df_room['売上'])

    # 連携データの名寄せ（統一店名への変換）
    mapping = dict(zip(df_tags['媒体側施設名'], df_tags['統一施設名']))
    df_link['統一店名'] = df_link['店名'].map(mapping).fillna(df_link['店名'])

    # 1. 全体元データから「いいアプリ」と「外部媒体」を日別に集計
    df_room['is_internal'] = df_room['媒体名'].str.contains('いいアプリ|会議室')
    daily_room = df_room.groupby(['dt', '統一店名', 'is_internal'])['売上'].sum().unstack(fill_value=0).reset_index()
    # カラム名が存在しない場合の補完
    if True not in daily_room.columns: daily_room[True] = 0
    if False not in daily_room.columns: daily_room[False] = 0
    daily_room.rename(columns={True: '設備予約_内部', False: '外部媒体売上_部屋'}, inplace=True)

    # 2. 連携データからベース売上（月額や分配金など、会議室以外の売上）を抽出
    base_cols = ['従量課金', '月額契約', '分配金', 'その他']
    for c in base_cols:
        if c in df_link.columns: df_link[c] = clean_num(df_link[c])
        else: df_link[c] = 0
    df_link['ベース売上'] = df_link[base_cols].sum(axis=1)
    daily_base = df_link.groupby(['dt', '統一店名'])['ベース売上'].sum().reset_index()

    # 3. データの結合と月ごとの累計計算
    df_merged = pd.merge(daily_base, daily_room, on=['dt', '統一店名'], how='outer').fillna(0)
    df_merged['年月'] = df_merged['dt'].dt.strftime('%Y-%m')

    # 店名と日付でソート（累計計算用）
    df_merged = df_merged.sort_values(['統一店名', 'dt'])

    # 店舗・月ごとに累計（進捗）を算出
    # 店舗全体売上 = 累計(ベース売上 + 自社アプリ予約)
    df_merged['累計_店舗全体'] = df_merged.groupby(['統一店名', '年月'])['ベース売上'].cumsum() + \
                               df_merged.groupby(['統一店名', '年月'])['設備予約_内部'].cumsum()
    # 外部媒体売上 = 累計(他社サイト予約)
    df_merged['累計_外部'] = df_merged.groupby(['統一店名', '年月'])['外部媒体売上_部屋'].cumsum()

    # 4. マイルストーン（5日刻み + 月末）の抽出
    def get_milestone_label(d):
        day = d.day
        # 5, 10, 15, 20, 25 の場合
        if day in [5, 10, 15, 20, 25]: return True
        # 月末日の場合（30日がない月や、31日まである月もカバー）
        if d.is_month_end: return True
        return False

    df_result = df_merged[df_merged['dt'].apply(get_milestone_label)].copy()

    # 5. カラム整形と並び替え
    df_result['年月日'] = df_result['dt'].dt.strftime('%Y/%m/%d')
    output = df_result[['統一店名', '年月日', '累計_店舗全体', '累計_外部']].copy()
    output.columns = ['店名', '年月日', '店舗全体売上', '外部媒体売上']

    # 【ご要望】店舗ごとにまとめたうえで、日付を昇順にする
    output = output.sort_values(['店名', '年月日'])

    # CSV出力とダウンロード
    out_name = f"店舗別_売上進捗レポート_{datetime.now().strftime('%Y%m%d')}.csv"
    output.to_csv(out_name, index=False, encoding='utf-8-sig')
    files.download(out_name)

    print(f"\n✅ 集計が完了しました！")
    print(f"ファイル名: {out_name}")
    print(f"集計期間: {output['年月日'].min()} ～ {output['年月日'].max()}")

if __name__ == '__main__':
    try:
        main_progress_tool_v2()
    except Exception as e:
        print(f"\n❌ エラーが発生しました: {e}")

以下の3つのCSVファイルをアップロードしてください。
1. 会議室シート_改 - 全体元データ.csv
2. 会議室シート_改 - タグ設置リスト.csv
3. いいアプリ売上データ置き場 - 連携.csv
--------------------------------------------------


Saving 会議室シート_改 - 全体元データ.csv to 会議室シート_改 - 全体元データ.csv
Saving 会議室シート_改 - タグ設置リスト.csv to 会議室シート_改 - タグ設置リスト.csv
Saving いいアプリ売上データ置き場 - 連携.csv to いいアプリ売上データ置き場 - 連携.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ 集計が完了しました！
ファイル名: 店舗別_売上進捗レポート_20260129.csv
集計期間: 2024/12/05 ～ 2026/01/25
